In [3]:
import sympy
from sympy import symbols, IndexedBase, Indexed, Idx, preorder_traversal
from outputC import *

Declaring indices and grid variables.  "var_names" will be used later for string replacements when generating AMReX block.

In [4]:
Nx, Ny, Nz, Nn= symbols('Nx Ny Nz Nn', integer=True)
i = Idx('i', Nx)
j = Idx('j', Ny)
k = Idx('k', Nz)
n = Idx('n', Nn)
dx, dy, dz = symbols('dx dy dz')
phi = IndexedBase('phi')
pi = IndexedBase('pi')
var_names = ['phi','pi']

These functions implement finite-differencing on grid variables. These are courtesy of Don.

In [5]:
def shift(E, idx_shift):
    # This function takes a generic Sympy expression and
    # returns a new Sympy expression where every Sympy Indexed
    # object in E has been shifted by idx_shift.
    # - idx_shift should be of length D, the dimension of E
    
    def shift_indexed(S, idx_shift):
        # This function returns a new IndexedBase object with shifted indices
        # - S should be a Sympy Indexed object
        # - idx_shift should be a tuple or list of index offsets to apply
        # - idx_shift should be of length D, the dimension of S
        base = S.base
        indices = [si + di for si, di in zip(S.indices, idx_shift)]
        return base[indices]

    return E.replace(lambda expr: type(expr) == Indexed, lambda expr: shift_indexed(expr, idx_shift))

def Dc(E, direction):
    assert(direction == 'x' or direction == 'y' or direction == 'z')
    if direction == 'x':
        shift_hi = (1, 0, 0, 0)
        shift_lo = (-1, 0, 0, 0)
        delta = dx
    elif direction == 'y':
        shift_hi = (0, 1, 0, 0)
        shift_lo = (0, -1, 0, 0)
        delta = dy
    elif direction == 'z':
        shift_hi = (0, 0, 1, 0)
        shift_lo = (0, 0, -1, 0)
        delta = dz
    return (shift(E, shift_hi) - shift(E, shift_lo))/(2 * delta)

def Dc2(E, direction):
    assert(direction == 'x' or direction == 'y' or direction == 'z')
    if direction == 'x':
        shift_hi = (1, 0, 0, 0)
        shift_lo = (-1, 0, 0, 0)
        delta = dx*dx
    elif direction == 'y':
        shift_hi = (0, 1, 0, 0)
        shift_lo = (0, -1, 0, 0)
        delta = dy*dy
    elif direction == 'z':
        shift_hi = (0, 0, 1, 0)
        shift_lo = (0, 0, -1, 0)
        delta = dz*dz
    return (shift(E, shift_hi) - 2*E + shift(E, shift_lo))/(delta)



Here we define symbolic names that will be used in the equations of motion. They will be used when expanding out the equations of motion for AMReX code blocks.

In [6]:
PHI = phi[i,j,k,n]
PHI_dx = Dc(phi[i,j,k,n],'x')
PHI_dxx = Dc2(phi[i,j,k,n],'x')
PHI_dy = Dc(phi[i,j,k,n],'y')
PHI_dyy = Dc2(phi[i,j,k,n],'y')

PI = pi[i,j,k,n]
PI_dx = Dc(pi[i,j,k,n],'x')
PI_dxx = Dc2(pi[i,j,k,n],'x')
PI_dy = Dc(pi[i,j,k,n],'y')
PI_dyy = Dc2(pi[i,j,k,n],'y')

Symbolic forms of the equations of motion.

In [7]:
RHS_PHI = PI
RHS_PI = PHI_dxx+PHI_dyy

Here we define the AMReXcode generator that takes the RHS expressions above and returns a string of compilable code to be inserted into the AMReX time stepper.

In [16]:
def AMReXcode(expr,varnames):
    str_expr = str(expr)
    for name in varnames:
        str_expr = str_expr.replace(name,name+"_old_fab")
    str_expr = str_expr.replace("[","(").replace("]",")")
    str_expr = str_expr.replace("dx**2","(dx[0]*dx[0])")
    str_expr = str_expr.replace("dy**2","(dx[1]*dx[1])")
    str_expr = str_expr+";"
    return str_expr

In [18]:
print(AMReXcode(RHS_PI,var_names))

(phi_old_fab(i, j + 1, k, n) + phi_old_fab(i, j - 1, k, n) - 2*phi_old_fab(i, j, k, n))/(dx[1]*dx[1]) + (phi_old_fab(i + 1, j, k, n) + phi_old_fab(i - 1, j, k, n) - 2*phi_old_fab(i, j, k, n))/(dx[0]*dx[0]);
